In [1]:
import pandas as pd
import pywt

In [2]:
pd.__version__

'1.0.1'

In [3]:
pywt.__version__

'1.1.1'

# Import data

In [14]:
data = pd.read_csv('SP-280-Leste.csv', sep=',',header=None)
data.head()

,0,1,2,3
0,1,01/09/2011 00:00,37,89.0
1,2,01/09/2011 00:05,48,82.0
2,3,01/09/2011 00:10,26,85.0
3,4,01/09/2011 00:15,47,81.0
4,5,01/09/2011 00:20,52,82.0


## Clean and organize dataset

+ ### Organize the dataset

    Organize the columns, remove some not useful data, rename the columns and convert each column to your correct unit.

+ ### Date-time columns

    Working with date-time date requires to change the index DataFrame to the column that contain Date-Time values


In [15]:
'''Organize'''
# Drop the column '0'
data = data.drop(columns=0)

# Rename columns
data = data.rename(columns={0:"ID", 1:"DateTime",2:"Flow[veh/h.lane]", 3:"Speed[km/h]" })

# Calculate the Flow rate
data["Flow[veh/h.lane]"] = data["Flow[veh/h.lane]"] * 60 / 5 / 2 # two lanes

# Calculate the Density
data["Density[veh/km.lane]"] = data["Flow[veh/h.lane]"] / data['Speed[km/h]']

# Convert to Datetime
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')

'''Date-time column index'''
data.set_index('DateTime',inplace=True)
data.head()

,Flow[veh/h.lane],Speed[km/h],Density[veh/km.lane]
DateTime,,,
2011-09-01 00:00:00,222.0,89.0,2.494382
2011-09-01 00:05:00,288.0,82.0,3.512195
2011-09-01 00:10:00,156.0,85.0,1.835294
2011-09-01 00:15:00,282.0,81.0,3.481481
2011-09-01 00:20:00,312.0,82.0,3.804878


+ ### Missing values

    Verifying some missing values in dataset


In [28]:
data.isna?